## Generate and evaluate Dai'sresults

In [1]:
from ModelLearning.genPOMDP import *
from os import system

diffs = [[0,1], #5 states
         [0, 0.33, 0.66, 1], #9 states
         [0, 0.2, 0.4, 0.6, 0.8, 1], # 13 states
         [0, 0.15, 0.3, 0.45, 0.6, 0.75, 0.9, 1] #17 states
        ]
#wrong_answer_costs = [-10,-500,-1000,-5000,-10000,-15000]

#filename, wronganswercost,createjobcost,distributionavg,numpools
for wrong_cost in [-500]:
    for diff in diffs:
        num_st = len(diff)
        genPOMDP('log/pomdp/rl.pomdp', wrong_cost, [1], [1], 1, diff)
        system(f"/Users/pmaglione/Repos/adaptive-pomdp-solutions/WorkerPoolSelection/ModelLearning/zmdp-1.1.7/bin/darwin18/zmdp solve log/pomdp/rl.pomdp -o ModelLearning/Policies/s{num_st}-{wrong_cost}.policy -t 300")
    
    

### Generate 24states with unclassified pomdp

In [1]:
from ModelLearning.genPOMDP import *
from os import system
from ModelLearning.utils import *
diff = getDifficulties(0.1)
num_st = len(diff)
wrong_cost = -500
genPOMDP('log/pomdp/unclassified.pomdp', wrong_cost, [1], [1], 1, diff)
)

In [17]:
for diff_v in diffs:
    for c in [0,1]:
        for diff in diff_v:
            print(f"({diff},{c})", end="", flush=True)
    print("\n")
    

(0,0)(1,0)(0,1)(1,1)

(0,0)(0.33,0)(0.66,0)(1,0)(0,1)(0.33,1)(0.66,1)(1,1)

(0,0)(0.2,0)(0.4,0)(0.6,0)(0.8,0)(1,0)(0,1)(0.2,1)(0.4,1)(0.6,1)(0.8,1)(1,1)

(0,0)(0.15,0)(0.3,0)(0.45,0)(0.6,0)(0.75,0)(0.9,0)(1,0)(0,1)(0.15,1)(0.3,1)(0.45,1)(0.6,1)(0.75,1)(0.9,1)(1,1)



## Run experiment

In [10]:
from os import system

sts = [5, 9, 13, 17]
for st in sts:
    system(f"/bin/bash run_experiment.sh 300 1000 0.5 {st}")

In [ ]:
system(f"/bin/bash run_experiment.sh 300 1000 0.5 24")

## Process results and show

In [9]:
import algorithms_utils as alg_utils
import pandas as pd
import numpy as np

def round_to_3(value):
    return round(value, 3)

WP0_num,WP0_mean,WP0_stddev,WP1_num,WP1_mean,WP1_stddev = 1000,4.000000,0.300000,1000,4.000000,0.300000
#name = "%d,%.2f,%.2f,%d,%.2f,%.2f" % (WP0_num,WP0_mean,WP0_stddev,WP1_num,WP1_mean,WP1_stddev)

#sts = [5, 9, 13, 17]
sts = [23]

for st in sts:
    print(f"Num States: {st}")
    RELATIVEPATH = f"Experiments/diff_rates - 100,1.00,0.20,100,1.00,0.20"

    NUMBER_OF_REPETITIONS = 5
    loss_ratio = 5
    total_cost_mean = 0.
    initial_line = 8
    #wrong_answer_costs = [10,500,1000,5000,10000,15000]
    wrong_answer_costs = [500]
    NUMBER_OF_ITEMS = 1000
    worker_num = 100
    data_bal = 0.5

    for wrongAnswerCost in wrong_answer_costs:
        losses = []
        recalls = []
        precisions = []
        costs = []

        for iteration in range(NUMBER_OF_REPETITIONS):
            iteration_results_file = open(RELATIVEPATH + "/%d/Unstarred/%d" % (wrongAnswerCost,iteration),'r')
            gt_vals = []
            classification_vals = []
            iteration_cost = []
            for i, line in enumerate(iteration_results_file):
                if (initial_line - 2 < i < (initial_line + NUMBER_OF_ITEMS - 1)):
                    item_id, y_val, gt_val, item_cost, item_difficulty, item_real_difficulty,  = line.rstrip().split(",")
                    classification_vals.append(int(y_val))
                    gt_vals.append(int(gt_val))
                    iteration_cost.append(float(item_cost))
            # end for results file

            costs.append(np.mean(iteration_cost))    
            loss, recall, precision = alg_utils.Metrics.compute_metrics(classification_vals, gt_vals, loss_ratio)
            losses.append(loss)
            recalls.append(recall)
            precisions.append(precision)
        # end for iterations

        print(f"Results for cost: {wrongAnswerCost}:")
        print(f"Cost mean: {round_to_3(np.mean(costs))} - Std: {round_to_3(np.std(costs))}")
        print(f"Recall mean: {round_to_3(np.mean(recalls))} - Std: {round_to_3(np.std(recalls))}")
        print(f"Precision mean: {round_to_3(np.mean(precisions))} - Std: {round_to_3(np.std(precisions))}")
        print(f"Loss mean: {round_to_3(np.mean(losses))} - Std: {round_to_3(np.std(losses))}")

        norm_cost = round_to_3(np.mean(costs)) * NUMBER_OF_ITEMS * 0.05
        norm_cost_std = round_to_3(np.std(costs)) * NUMBER_OF_ITEMS * 0.05

        print("  ---  ")

Num States: 23
Results for cost: 500:
Cost mean: 10.127 - Std: 0.202
Recall mean: 0.876 - Std: 0.027
Precision mean: 0.875 - Std: 0.018
Loss mean: 0.381 - Std: 0.067
  ---  


# Dai's Results over real-world datasets

In [6]:
import algorithms_utils as alg_utils
import pandas as pd
import numpy as np

def round_to_3(value):
    return round(value, 3)

WP0_num,WP0_mean,WP0_stddev,WP1_num,WP1_mean,WP1_stddev = 1000,4.000000,0.300000,1000,4.000000,0.300000
#name = "%d,%.2f,%.2f,%d,%.2f,%.2f" % (WP0_num,WP0_mean,WP0_stddev,WP1_num,WP1_mean,WP1_stddev)

datasets = ['BarzanMozafari', 'RTE', 'SpamCF', 'TEMP', 'WVSCM']
RELATIVEPATH = "Experiments/Results/RealWorld/"

NUMBER_OF_REPETITIONS = 6
loss_ratio = 5
total_cost_mean = 0.
initial_line = 8
wrong_answer_costs = [500]

gammas_dist = [(4.,0.42),(4.,0.15),(4.,0.4),(4.,0.14),(4.,0.5)]

for key,name in enumerate(datasets):
    print(name)

    for wrongAnswerCost in wrong_answer_costs:
        ground_truth, workers_accuracy = alg_utils.get_real_dataset_data(name)
        NUMBER_OF_ITEMS = len(ground_truth)
        
        data_bal = sum(ground_truth) / NUMBER_OF_ITEMS
        worker_num = len(workers_accuracy)
        
        losses = []
        recalls = []
        precisions = []
        costs = []

        for iteration in range(NUMBER_OF_REPETITIONS):
            iteration_results_file = open(RELATIVEPATH + "%s/%d/Unstarred/%d" % (name,wrongAnswerCost,iteration),'r')
            gt_vals = []
            classification_vals = []
            iteration_cost = 0.
            for i, line in enumerate(iteration_results_file):
                if (initial_line - 1 < i < (initial_line + NUMBER_OF_ITEMS - 1)):
                    item_id, y_val, gt_val, item_cost, item_difficulty, item_real_difficulty,  = line.rstrip().split(",")
                    classification_vals.append(int(y_val))
                    gt_vals.append(int(gt_val))
                    iteration_cost += float(item_cost)
            # end for results file

            costs.append(iteration_cost / NUMBER_OF_ITEMS)    
            loss, recall, precision = alg_utils.Metrics.compute_metrics(classification_vals, gt_vals, loss_ratio)
            losses.append(loss)
            recalls.append(recall)
            precisions.append(precision)
        # end for iterations

        print(f"Results for cost: {wrongAnswerCost}:")
        print(f"Cost mean: {round_to_3(np.mean(costs))} - Std: {round_to_3(np.std(costs))}")
        print(f"Recall mean: {round_to_3(np.mean(recalls))} - Std: {round_to_3(np.std(recalls))}")
        print(f"Precision mean: {round_to_3(np.mean(precisions))} - Std: {round_to_3(np.std(precisions))}")
        print(f"Loss mean: {round_to_3(np.mean(losses))} - Std: {round_to_3(np.std(losses))}")
        
        norm_cost = round_to_3(np.mean(costs)) * NUMBER_OF_ITEMS * 0.05
        norm_cost_std = round_to_3(np.std(costs)) * NUMBER_OF_ITEMS * 0.05
        
        shape,scale = gammas_dist[key]
        norm_gamma = f"{shape}-{scale}"
        
        #loss_ratio,cost_ratio,wrong_answer_cost,name,workers_num,gamma_dist,data_bal,cost,cost_std,loss,loss_std,recall,recall_std,precision,precision_std
        print(f"5,0.05,500,{name},{worker_num},{norm_gamma},{data_bal},{norm_cost},{norm_cost_std},{round_to_3(np.mean(losses))},{round_to_3(np.std(losses))},{round_to_3(np.mean(recalls))},{round_to_3(np.std(recalls))},{round_to_3(np.mean(precisions))},{round_to_3(np.std(precisions))}")
        
        print("  ---  ")
    

BarzanMozafari
Results for cost: 500:
Cost mean: 10.726 - Std: 0.342
Recall mean: 0.914 - Std: 0.015
Precision mean: 0.907 - Std: 0.015
Loss mean: 0.264 - Std: 0.041
5,0.05,500,BarzanMozafari,83,4.0-0.42,0.491,536.3000000000001,17.1,0.264,0.041,0.914,0.015,0.907,0.015
  ---  
RTE
Results for cost: 500:
Cost mean: 3.743 - Std: 0.042
Recall mean: 0.98 - Std: 0.006
Precision mean: 0.981 - Std: 0.006
Loss mean: 0.061 - Std: 0.012
5,0.05,500,RTE,164,4.0-0.15,0.5,149.72,1.6800000000000002,0.061,0.012,0.98,0.006,0.981,0.006
  ---  
SpamCF
Results for cost: 500:
Cost mean: 10.355 - Std: 0.274
Recall mean: 0.92 - Std: 0.025
Precision mean: 0.974 - Std: 0.019
Loss mean: 0.3 - Std: 0.079
5,0.05,500,SpamCF,150,4.0-0.4,0.3069306930693069,52.292750000000005,1.3837000000000002,0.3,0.079,0.92,0.025,0.974,0.019
  ---  
TEMP
Results for cost: 500:
Cost mean: 3.602 - Std: 0.051
Recall mean: 0.978 - Std: 0.005
Precision mean: 0.985 - Std: 0.002
Loss mean: 0.07 - Std: 0.019
5,0.05,500,TEMP,76,4.0-0.14,0.43

# Test compute better values for each action

In [182]:
from ModelLearning.utils import *
from copy import deepcopy

def myfindBestValue(action, hyperplanes, beliefs):
    bestValue = -129837198273981231
    bestHyperplane = []
    amount = 0

    for hyperplane in hyperplanes:
        dontUse = False
        for (b, entry) in zip(beliefs, hyperplane):
            if b != 0 and entry == '*':
                dontUse = True
                break
        if dontUse:
            amount = amount + 1
            continue
        
        value = dot(beliefs, hyperplane)
        if value > bestValue:
            bestHyperplane = hyperplane
            bestValue = value
     
    return bestValue

def myfindBestAction(actions, policy, beliefState):
    bestValue = -1230981239102938019
    bestAction = 0  #Assume there is at least one action
    for action in actions:
        if action in policy:
            value = myfindBestValue(action, policy[action], beliefState)
            if value > bestValue:
                bestValue = value
                bestAction = action
    return bestAction

def findValues(actions, policy, beliefState):
    bestValue = [-1230981239102938019] * len(actions)

    for action in actions:
        if action in policy:
            value = myfindBestValue(action, policy[action], beliefState)
            if value > bestValue[action]:
                bestValue[action] = value

    return bestValue

numberOfWorkerPools = 1
numStates = 23
numberOfProblems = 100
actions = range(0, numberOfWorkerPools+2)

belief = [1 for i in range(numStates)]  # init , equivalent to [1] * numStates
belief[numStates-1] = 0  # last states = 0, terminating state
belief = normalize(belief)
beliefs = [deepcopy(belief) for i in range(numberOfProblems)]

#policies = {'frtdp':'/Users/pmaglione/Documents/pomdp_solve_test/frtdp.policy',
#            'hsvi':'/Users/pmaglione/Documents/pomdp_solve_test/hsvi.policy'}

policies = {'10': '/Users/pmaglione/Repos/adaptive-pomdp-solutions/WorkerPoolSelection/ModelLearning/Policies/W1_COST-10.policy',
           '1000': '/Users/pmaglione/Repos/adaptive-pomdp-solutions/WorkerPoolSelection/ModelLearning/Policies/W1_COST-1000.policy'}

values = {'frtdp':0, 'hsvi':0}

for key,path in policies.items():
    policy = readPolicy(path, numStates)
    beliefState = beliefs[0]
    #print(beliefState)
    print(f"Strategy: {key}")
    key_vals = findValues(actions, policy, beliefState)
    values[key] = key_vals
    print(key_vals)

Strategy: 10
[-3.4997499999999997, -5.000000000000002, -5.000000000000002]
Strategy: 1000
[-88.09397909090907, -499.9999999999999, -409.090909090909]


In [142]:
numStates = 22
numberOfProblems = 10
belief = [1 for i in range(numStates)]  # init , equivalent to [1] * numStates
belief[numStates-1] = 0  # last states = 0, terminating state
belief = normalize(belief)
beliefs = [deepcopy(belief) for i in range(numberOfProblems)]


gammas = [0.628980,
0.580959,
1.281037,
0.987592,
1.373258,
0.801062,
1.488326,
1.523445,
1.448263,
1.126245,
1.046380,
1.418695,
1.118415,
1.142464,
1.032052,
0.860525,
1.418732]

gammas = [1.281037] * 100

difficulties = getDifficulties(0.1)
gen_belief = beliefs[0]
observation = 1

num = 0

for gamma in gammas:
    gen_belief = updateBelief(gen_belief, observation, difficulties, gamma)
    action = findBestAction([0,1,2], policy, gen_belief)
    print(action)
    print(gen_belief)
    #print(gen_belief.index(max(gen_belief)))
    print("---")
    num = num + 1
    if (action != 0):
        print(num)
        break
    observation = np.random.binomial(1, 0.75)


0
[0.0, 0.0060123097415043, 0.011839454877812516, 0.01746500369237076, 0.022868532129299814, 0.028023827666600165, 0.03289575883800424, 0.03743421992492823, 0.04156041989520074, 0.04512592461901333, 0.04761904761904763, 0.09523809523809526, 0.08922578549659095, 0.08339864036028273, 0.07777309154572448, 0.07236956310879543, 0.06721426757149508, 0.062342336400091014, 0.057803875313167025, 0.05367767534289451, 0.05011217061908192, 0.0, 0.0]
---
0
[0.0, 0.01649468179411477, 0.030360078668685404, 0.04176478596555842, 0.05088695399564624, 0.05791634407769344, 0.06305727498908088, 0.06653313053722262, 0.06859395455811654, 0.06953149123097968, 0.06972260836580384, 0.0, 0.01649468179411477, 0.0303600786686854, 0.04176478596555841, 0.05088695399564624, 0.05791634407769343, 0.06305727498908088, 0.06653313053722262, 0.06859395455811655, 0.06953149123097971, 0.0, 0.0]
---
0
[0.0, 0.002082593856709167, 0.007548382411179521, 0.015317864963090597, 0.02443790879064439, 0.0340837905557688, 0.04356065513

In [152]:
b = [0.045454545454545456,
 0.045454545454545456,
 0.045454545454545456,
 0.045454545454545456,
 0.045454545454545456,
 0.045454545454545456,
 0.045454545454545456,
 0.045454545454545456,
 0.045454545454545456,
 0.045454545454545456,
 0.045454545454545456,
 0.045454545454545456,
 0.045454545454545456,
 0.045454545454545456,
 0.045454545454545456,
 0.045454545454545456,
 0.045454545454545456,
 0.045454545454545456,
 0.045454545454545456,
 0.045454545454545456,
 0.045454545454545456,
 0.045454545454545456,
0.]
p0 = [-3.9995, -4.97644, -6.20159, -7.90205, -10.5238, -15.1055, -24.4401, -46.3647, -98.0642, -192.416, -291.557, -3.9995, -4.97644, -6.20159, -7.90204, -10.5233, -15.0967, -24.3655, -46.0182, -97.2571, -191.371, -290.521, 0.0]
p1 = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -500.0, -500.0, -500.0, -500.0, -500.0, -500.0, -500.0, -500.0, -500.0, -500.0, -500.0, 0.0]
p2 = [-500.0, -500.0, -500.0, -500.0, -500.0, -500.0, -500.0, -500.0, -500.0, -500.0, -500.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0]

print(dot(b, p0))
print(dot(b, p1))
print(dot(b, p2))

-63.626511363636375
-249.49999999999994
-249.50000000000009


In [180]:
from ModelLearning.utils import *
from copy import deepcopy

def myfindBestValue(action, hyperplanes, beliefs):
    bestValue = -129837198273981231
    bestHyperplane = []
    amount = 0

    for hyperplane in hyperplanes:
        dontUse = False
        for (b, entry) in zip(beliefs, hyperplane):
            if b != 0 and entry == '*':
                dontUse = True
                break
        if dontUse:
            amount = amount + 1
            continue
        
        value = dot(beliefs, hyperplane)
        if value > bestValue:
            bestHyperplane = hyperplane
            bestValue = value
     
    return bestValue

def myfindBestAction(actions, policy, beliefState):
    bestValue = -1230981239102938019
    bestAction = 0  #Assume there is at least one action
    for action in actions:
        if action in policy:
            value = myfindBestValue(action, policy[action], beliefState)
            if value > bestValue:
                bestValue = value
                bestAction = action
    return bestAction

def findValues(actions, policy, beliefState):
    bestValue = [-1230981239102938019] * len(actions)

    for action in actions:
        if action in policy:
            value = myfindBestValue(action, policy[action], beliefState)
            if value > bestValue[action]:
                bestValue[action] = value

    return bestValue

numberOfWorkerPools = 1
numStates = 23
numberOfProblems = 100
actions = range(0, numberOfWorkerPools+2)

belief = [1 for i in range(numStates)]  # init , equivalent to [1] * numStates
belief[numStates-1] = 0  # last states = 0, terminating state
belief = normalize(belief)
beliefs = [deepcopy(belief) for i in range(numberOfProblems)]

def myReadPolicy(path, numStates):
    f = open(path, "r")
    alpha_vectors = {0: [], 1:[], 2:[]}

    action = -1
    for line in f:
        if len(line) == 2:
            action = int(line)
        elif len(line) > 2:
            alpha_vectors[action].append([float(x) for x in line.split()])
    
    return alpha_vectors
    

path = '/Users/pmaglione/Documents/pomdp_solve_test/pomdp-solve'
values = {}

methods = ['enum','twopass','witness','incprune']
beliefState = beliefs[0]
print(f"BeliefState: {beliefState}")
    
#for method in methods:
file = f"/Users/pmaglione/Repos/adaptive-pomdp-solutions/WorkerPoolSelection/ModelLearning/Policies/W1_COST-10.policy"

policy = myReadPolicy(file, numStates)

print(f"Strategy: {method}")
key_vals = findValues(actions, policy, beliefState)
values[key] = key_vals
print(key_vals)
    

BeliefState: [0.045454545454545456, 0.045454545454545456, 0.045454545454545456, 0.045454545454545456, 0.045454545454545456, 0.045454545454545456, 0.045454545454545456, 0.045454545454545456, 0.045454545454545456, 0.045454545454545456, 0.045454545454545456, 0.045454545454545456, 0.045454545454545456, 0.045454545454545456, 0.045454545454545456, 0.045454545454545456, 0.045454545454545456, 0.045454545454545456, 0.045454545454545456, 0.045454545454545456, 0.045454545454545456, 0.045454545454545456, 0.0]


KeyError: -1

# Test 2 states POMDP policy

In [130]:
from ModelLearning.utils import *
from copy import deepcopy
import numpy as np
import random
from truth_finder import expectation_maximization

#metrics
class Metrics:

    @staticmethod
    #k penalization for false negatives
    def compute_metrics(items_classification, gt, lr = 1):
        # FP == False Inclusion
        # FN == False Exclusion
        fp = fn = tp = tn = 0.
        for i in range(len(gt)):
            gt_val = gt[i]
            cl_val = items_classification[i]

            if gt_val and not cl_val:
                fn += 1
            if not gt_val and cl_val:
                fp += 1
            if gt_val and cl_val:
                tp += 1
            if not gt_val and not cl_val:
                tn += 1
                        

        recall = tp / (tp + fn)
        precision = tp / (tp + fp)
        loss = (fp + (fn * lr)) / len(gt)
        
        return loss, recall, precision
#end

def simulate_workers(workers_num, cheaters_prop, fixed_acc, workers_acc, base_acc = .5):
    workers = {}
    for i in range(workers_num):
        if (fixed_acc == False):
            if np.random.binomial(1, cheaters_prop):
                # worker_type is 'rand_ch'
                worker_acc_pos = worker_acc_neg = 0.5
            else:
                # worker_type is 'worker'
                worker_acc_pos = base_acc + (np.random.beta(1, 1) * (1 - base_acc))
                #worker_acc_neg = worker_acc_pos + 0.1 if worker_acc_pos + 0.1 <= 1. else 1.
                worker_acc_neg = worker_acc_pos
        else:
            worker_acc_pos = workers_acc
            worker_acc_neg = worker_acc_pos

        workers[i] = [worker_acc_pos, worker_acc_neg]

    return workers

def get_random_worker_accuracy(workers_accuracy, item_id, votes):
    item_votes = votes[item_id].copy()
    worker_ids_used = item_votes.keys()
    workers_ids_range = workers_accuracy.keys()
    workers_ids_unused = [val for val in workers_ids_range if val not in worker_ids_used]
    
    if (len(workers_ids_unused) == 0):
        used = len(worker_ids_used)
        ranges = len(workers_ids_range)
        unu = len(workers_ids_unused)
        print(f'used: {used}')
        print(f'workers: {ranges}')
        print(f'unused: {unu}')
        raise ValueError("Unused empty!?")
    
    selected_worker_id = np.random.choice(workers_ids_unused)
    worker_acc_pos = workers_accuracy[selected_worker_id][0]
    worker_acc_neg = workers_accuracy[selected_worker_id][1]

    return {'worker_id': selected_worker_id, 'acc_pos':worker_acc_pos, 'acc_neg': worker_acc_neg}


def get_worker_vote(workers_accuracy, i, gt, votes):
    worker_data = get_random_worker_accuracy(workers_accuracy, i, votes)
    worker_id, worker_acc_pos, worker_acc_neg = worker_data['worker_id'], worker_data['acc_pos'], worker_data['acc_neg']

    if (gt[i]):
        worker_acc = worker_acc_pos
    else:
        worker_acc = worker_acc_neg

    if np.random.binomial(1, worker_acc):
        vote = gt[i]
    else:
        vote = 1 - gt[i]

    return (worker_id, vote)

def generate_gold_data(items_num, possitive_percentage):
    pos_items_number = int(round(((possitive_percentage * 100) * items_num) / 100))     
    gold_data = ([1] * pos_items_number) + ([0] * (items_num - pos_items_number))
    random.shuffle(gold_data)

    return gold_data

def findBestValue(action, hyperplanes, beliefs):
    bestValue = -129837198273981231
    bestHyperplane = []
    amount = 0

    for hyperplane in hyperplanes:
        dontUse = False
        for (b, entry) in zip(beliefs, hyperplane):
            if b != 0 and entry == '*':
                dontUse = True
                break
        if dontUse:
            amount = amount + 1
            continue
        
        value = dot(beliefs, hyperplane)
        if value > bestValue:
            bestHyperplane = hyperplane
            bestValue = value
     
    return bestValue

def findBestAction(actions, policy, beliefState):
    bestValue = -1230981239102938019
    bestAction = 0  #Assume there is at least one action
    for action in actions:
        if action in policy:
            value = findBestValue(action, policy[action], beliefState)
            if value > bestValue:
                bestValue = value
                bestAction = action
    return bestAction

def myReadPolicy(path, states, origin):
    if origin != 'zmdp':
        f = open(path, "r")
        alpha_vectors = {0: [], 1:[], 2:[]}

        action = -1
        for line in f:
            if len(line) == 2:
                action = int(line)
            elif len(line) > 2:
                strategy_vectors = [float(x) for x in line.split()]
                if len(strategy_vectors) == states:
                    alpha_vectors[action].append(strategy_vectors)
    else:
        alpha_vectors = readPolicy(path, states)
    
    return alpha_vectors

def update_belief(prevBelief, vote, acc, states_per_class):
    newBeliefs = []
    for i in range(0, 2):  # 0,1
        for j in range(0, states_per_class):
            state = (i * states_per_class) + j
            
            if vote == i:
                newBeliefs.append(acc * prevBelief[state])
            else:
                newBeliefs.append((1 - acc) * prevBelief[state])
                
    newBeliefs.append(0.0)
    normalize(newBeliefs)
    return newBeliefs

def input_adapter(responses):
    '''
    :param responses:
    :return: Psi
    '''
    Psi = [[] for _ in responses.keys()]
    i = 0
    for obj_id, obj_responses in responses.items():
        k = 0
        for worker_id, worker_response in obj_responses.items():
            Psi[i].append((k, worker_response[0]))
            k += 1
        i += 1
    return Psi


In [133]:
def run_exp(policy, items, states, num_actions, workers_accuracy, ground_truth, acc):
    belief = [1 for i in range(states)]  # init , equivalent to [1] * numStates
    belief[states-1] = 0  # last states = 0, terminating state
    belief = normalize(belief)
    beliefs = [deepcopy(belief) for i in range(items)]
    beliefState = beliefs[0]
    actions = range(0, num_actions)
    items_classification = {}
    votes = {i:{} for i in range(items)}

    states_per_class = int((states - 1) / 2)

    while len(items_classification) != items:
        for item_id, belief_state in enumerate(beliefs):
            if item_id not in items_classification.keys(): # if item is classified
                best_action = findBestAction(actions, policy, belief_state)

                #prevent
                if len(votes[item_id]) > 100:
                    mv = (sum(votes[item_id]) / len(votes[item_id])) > 0.5
                    if mv:
                        best_action = 2
                    else:
                        best_action = 1

                if best_action == 0:
                    worker_id, vote = get_worker_vote(workers_accuracy, item_id, ground_truth, votes)
                    votes[item_id][worker_id] = [vote]
                    
                    belief_state = update_belief(belief_state, vote, acc, states_per_class)
                    
                    beliefs[item_id] = belief_state
                elif best_action == 1: #submit zero
                    items_classification[item_id] = 0
                else: #submit one
                    items_classification[item_id] = 1
                    
        #end for
        if min([len(v) for a,v in votes.items()]) == 3:
            accs, p = expectation_maximization(max([len(v) for a,v in votes.items()]), items, input_adapter(votes))
            acc = np.mean(accs)
        
    #end while
    num_votes = sum([len(v) for k,v in votes.items()])

    return items_classification, num_votes

In [128]:
data_true_percentage = .5
states = 3
items = 100
num_actions = 3
lr = 5
acc = .75

In [141]:
def run_experiment(policy, data_true_percentage, states, items, num_actions, lr, acc):
    ground_truth = generate_gold_data(items, data_true_percentage)
    workers_accuracy = simulate_workers(1000, 0, False, acc, base_acc = .5)
    
    losses  = []
    recalls  = []
    precisions  = []
    votes_amount = []
    for _ in range(50):
        items_classification, num_votes = run_exp(policy, items, states, num_actions, workers_accuracy, ground_truth, acc)
        
        votes_amount.append(num_votes / items) #average
        
        loss,  recall, precision = Metrics.compute_metrics(items_classification, ground_truth, lr)
        losses.append(loss)
        recalls.append(recall)
        precisions.append(precision)

    print(f" Votes: {np.mean(votes_amount)} / {np.std(votes_amount)} \n Loss: {np.mean(losses)} / {np.std(losses)} \n Recall: {np.mean(recalls)} / {np.std(recalls)} \n Precision: {np.mean(precisions)} / {np.std(precisions)}")

In [143]:
policies = []
policies.append(myReadPolicy('/Users/pmaglione/Documents/pomdp_solve_test/3states/-500/test.alpha', states, 'pomdp-solve'))
policies.append(myReadPolicy('/Users/pmaglione/Documents/pomdp_solve_test/3states/-1000/test.alpha', states, 'pomdp-solve'))
policies.append(myReadPolicy('/Users/pmaglione/Documents/pomdp_solve_test/3states/-10/test.alpha', states, 'pomdp-solve'))

for policy in policies:
    run_experiment(policy, data_true_percentage, states, items, num_actions, lr, acc)
    print(" --- ")

 Votes: 9.6322 / 0.9893680609358683 
 Loss: 0.0238 / 0.03205557673790942 
 Recall: 0.992 / 0.01200000000000001 
 Precision: 0.9925161202942715 / 0.012344492992408751
 --- 
 Votes: 10.5534 / 1.190007747873937 
 Loss: 0.0068000000000000005 / 0.014891608375189026 
 Recall: 0.9980000000000001 / 0.006000000000000005 
 Precision: 0.9964705882352942 / 0.007533087338156308
 --- 
 Votes: 1.0 / 0.0 
 Loss: 0.7298 / 0.16922753913001276 
 Recall: 0.7568 / 0.06564876236457166 
 Precision: 0.7577381369027768 / 0.04809981928535244
 --- 


In [144]:
states = 23
policy = myReadPolicy('./ModelLearning/Policies/W1_COST500.policy', states, 'zmdp')
run_experiment(policy, data_true_percentage, states, items, num_actions, lr, acc)

 Votes: 3.1920000000000006 / 0.17577258034175863 
 Loss: 0.2926 / 0.08666741025322032 
 Recall: 0.9036 / 0.033392214661504556 
 Precision: 0.8992675825856574 / 0.040620210608958304


In [8]:
for i in range(22):
    '''
    print(f"T: 0 : {i} : {i} 1.000000")
    print(f"T: 1 : {i} : 22 1.000000")  
    print(f"T: 2 : {i} : 22 1.000000")
    print(f"T: 3 : {i} : 23 1.000000")
    '''
    print(f"O: 3: {i} : None 1.00000")

O: 3: 0 : None 1.00000
O: 3: 1 : None 1.00000
O: 3: 2 : None 1.00000
O: 3: 3 : None 1.00000
O: 3: 4 : None 1.00000
O: 3: 5 : None 1.00000
O: 3: 6 : None 1.00000
O: 3: 7 : None 1.00000
O: 3: 8 : None 1.00000
O: 3: 9 : None 1.00000
O: 3: 10 : None 1.00000
O: 3: 11 : None 1.00000
O: 3: 12 : None 1.00000
O: 3: 13 : None 1.00000
O: 3: 14 : None 1.00000
O: 3: 15 : None 1.00000
O: 3: 16 : None 1.00000
O: 3: 17 : None 1.00000
O: 3: 18 : None 1.00000
O: 3: 19 : None 1.00000
O: 3: 20 : None 1.00000
O: 3: 21 : None 1.00000


In [ ]:
print(f"T: 3 : {i} : 23 1.000000")  